# Vaccine Dashboard Update
Center for Human Dynamics in the Mobile Age (HDMA) at San Diego State University <br>
Jessica Embury

### IMPORTS

In [65]:
#import modules
#import arcgis
from arcgis.gis import GIS
from arcgis import geometry
from arcgis.features import GeoAccessor, GeoSeriesAccessor
from arcgis.features import FeatureLayerCollection
from arcgis.features import FeatureLayer
#from arcgis.mapping import WebMap

from copy import deepcopy
import json
import numpy as np
import os
import pandas as pd
import requests
import sys
import tabula
import time
import webbrowser

#ignore pandas slice warning
pd.options.mode.chained_assignment = None  # default='warn'

### DATE, FILE PATHS, & AGOL IDS

In [152]:
###########################
###SET DATA THROUGH DATE###
###########################
most_recent_date = '02/27/2021'  # Change date daily using 'MM/DD/YYYY' format


##########################
###SET PATHS IN AND OUT###
##########################
#PATHS IN
pdf_path = 'C:/Users/jesse/Dropbox/Mapping-Vulearable-Pop-Tasks/SD-County-Data/{}/COVID-19 Vaccine Report by Zipcode.pdf'.format(most_recent_date.replace('/', '-'))

#additional needed zip info - community name and population
zips_info_path = './data/zip_coords_pop.csv'

#paths out
path_out = './data/sd_zip_vaccines.csv' #feature overwrite
path_out2 = './data/sd_zip_vaccines_alldates.csv' # date columns
path_out3 = '../covid_data/vaccines/sd_zip_vaccines_{}.csv'.format(most_recent_date.replace('/', '')) # overwrite data archive
path_db = 'C:/Users/jesse/Dropbox/Mapping-Vulearable-Pop-Tasks/SD-County-Data/COVID-Data-Share-at-HDMA-Center/Vaccination/SD_Zip_COVID_Vaccine_{}.csv'.format(most_recent_date.replace('/', ''))


vacc_layer_id = 'c87b9c4b90b3497b826a20c5a1c1e517'
vacc_count_map_id = '3a5a90a0b43b42bb823ab4136b8d58e9'
vacc_dash = 'https://arcg.is/15ePOz'

### CONVERT VACCINE PDF TO CSV FILES

In [67]:
#read pdf
tables = tabula.read_pdf(pdf_path, pages = "all", multiple_tables = True)

In [68]:
tables[0].head()

,0,1,2,3,4,5,6,7,8
0,Zip Code,NaN,Count,"Rate per 1,000*",Zip Code,NaN,Count,NaN,"Rate per 1,000*"
1,NaN,91901,"3,263",181.4,92069,NaN,"5,897",NaN,115.2
2,NaN,91902,"4,712",267.3,92070,NaN,219,NaN,**
3,NaN,91905,228,**,92071,NaN,"9,207",NaN,158.2
4,NaN,91906,508,**,92075,NaN,"3,616",NaN,278.6


In [69]:
#format 1st column zips
t1 = tables[0][[1, 2, 3]]
#rename columns
t1 = t1.rename(columns = {1: 'Zip Code', 2: 'Vacc Count', 3: 'Rate Per 1000'})
# drop bad rows
t1 = t1.drop(t1.index[0])
t1.head()

,Zip Code,Vacc Count,Rate Per 1000
1,91901,"3,263",181.4
2,91902,"4,712",267.3
3,91905,228,**
4,91906,508,**
5,91910,"15,572",185.7


In [70]:
#format 2nd column zips
t2 = tables[0][[4, 6, 8]]
#rename columns
t2 = t2.rename(columns = {4: 'Zip Code', 6: 'Vacc Count', 8: 'Rate Per 1000'})
# drop bad rows
t2 = t2.drop(t2.index[[0, 56, 57]])
#t2.head()
t2.tail()

,Zip Code,Vacc Count,Rate Per 1000
51,92173,"6,174",227.8
52,92182,7,**
53,92259,7,**
54,92536,97,**
55,92672,479,**


In [71]:
# merge t1 and t2
t1 = t1.append(t2, ignore_index=True)
print(len(t1))

t1

112


,Zip Code,Vacc Count,Rate Per 1000
0,91901,"3,263",181.4
1,91902,"4,712",267.3
2,91905,228,**
3,91906,508,**
4,91910,"15,572",185.7
...,...,...,...
107,92173,"6,174",227.8
108,92182,7,**
109,92259,7,**
110,92536,97,**


In [72]:
#format numbers in counts column
t1 = t1.fillna(0)

for i, row in t1.iterrows():
    try:
        t1['Vacc Count'][i] = str(t1['Vacc Count'][i].replace(',', ''))
        t1['Vacc Count'][i] = int(t1['Vacc Count'][i])
    except:
        print(t1['Zip Code'][i], t1['Vacc Count'][i])
    
    try:
        t1['Rate Per 1000'][i] = str(t1['Rate Per 1000'][i].replace('**', '0'))
        t1['Rate Per 1000'][i] = float(t1['Rate Per 1000'][i])
    except:
        print(t1['Zip Code'][i], t1['Rate Per 1000'][i])

t1['Vacc Count'] = t1['Vacc Count'].astype(int)
print(type(t1['Vacc Count'][0]))

t1['Rate Per 1000'] = t1['Rate Per 1000'].astype(float)
print(type(t1['Rate Per 1000'][0]))

t1.head()

92096 0
92135 0
92140 0
<class 'numpy.int32'>
<class 'numpy.float64'>


,Zip Code,Vacc Count,Rate Per 1000
0,91901,3263,181.4
1,91902,4712,267.3
2,91905,228,0.0
3,91906,508,0.0
4,91910,15572,185.7


In [73]:
# supplemental zip code data
zips = pd.read_csv(zips_info_path)
zips.head()

,Zip,Community,2018_population,Latitude,Longitude
0,91901,Alpine,17885,32.805710,-116.695537
1,91902,Bonita,17375,32.671583,-117.015068
2,91905,Boulevard,2014,32.718365,-116.305469
3,91906,Campo,3686,32.660427,-116.469681
4,91910,Chula Vista,82682,32.636413,-117.065653


In [74]:
# merge vaccine info with zip code info
t1['Zip Code'] = t1['Zip Code'].astype(str)
zips['Zip'] = zips['Zip'].astype(str)

df = pd.merge(zips, t1, how = 'right', left_on = 'Zip', right_on = 'Zip Code')
del df['Zip Code']

# add data date
df['Date'] = most_recent_date
df

,Zip,Community,2018_population,Latitude,Longitude,Vacc Count,Rate Per 1000,Date
0,91901,Alpine,17885,32.805710,-116.695537,3263,181.4,02/27/2021
1,91902,Bonita,17375,32.671583,-117.015068,4712,267.3,02/27/2021
2,91905,Boulevard,2014,32.718365,-116.305469,228,0.0,02/27/2021
3,91906,Campo,3686,32.660427,-116.469681,508,0.0,02/27/2021
4,91910,Chula Vista,82682,32.636413,-117.065653,15572,185.7,02/27/2021
...,...,...,...,...,...,...,...,...
107,92173,San Ysidro,27741,32.554232,-117.047513,6174,227.8,02/27/2021
108,92182,San Diego,2222,32.776182,-117.073311,7,0.0,02/27/2021
109,92259,Ocotillo,0,32.765895,-116.153926,7,0.0,02/27/2021
110,92536,Aguanga,491,33.398676,-116.720547,97,0.0,02/27/2021


In [75]:
# get max count and rate for map updates
max_count = df['Vacc Count'].max()
max_rate = df['Rate Per 1000'].max()
print(max_count, max_rate)

15572 366.6


In [76]:
# save feature overwrite info
df.to_csv(path_out, index=False)
df.to_csv(path_out3, index=False)

In [77]:
# prep data for dated column csv
df2 = pd.read_csv(path_out2)
print(len(df2))
df2['Zip'] = df2['Zip'].astype(str)
df2 = df2.merge(df[['Zip', 'Vacc Count']], how = "right", on = "Zip")
print(len(df2))
df2 = df2.rename(columns = {'Vacc Count': most_recent_date})
df2.head()

112
112


,Zip,Community,Latitude,Longitude,02/23/2021,02/24/2021,02/25/2021,02/26/2021,02/27/2021
0,91901,Alpine,32.805710,-116.695537,3095,3167,3210,3238,3263
1,91902,Bonita,32.671583,-117.015068,4463,4531,4580,4650,4712
2,91905,Boulevard,32.718365,-116.305469,221,225,226,228,228
3,91906,Campo,32.660427,-116.469681,494,497,500,503,508
4,91910,Chula Vista,32.636413,-117.065653,14599,14889,15093,15321,15572


In [78]:
# save dated columns to csv
df2.to_csv(path_out2, index=False)
df2.to_csv(path_db, index=False)

### CONNECT TO AGOL ACCOUNT

In [79]:
gis = GIS("pro")

### FUNCTIONS

In [80]:
def get_map (map_id):
    '''
    GET MAP DATA FOR SYMBOLOGY CHANGES
    '''
    m = gis.content.get(map_id)
    data = m.get_data()
    #Include the below line for prettified JSON
    #print(json.dumps(data, indent=4, sort_keys=True))
    print(m)
    
    return data
    
def update_map (map_id, data):
    '''
    UPDATE MAP TO SAVE CHANGES
    '''
    m = gis.content.get(map_id)
    # Set the item_properties to include the desired update
    properties = {"text": json.dumps(data)}
    # 'Commit' the updates to the Item
    update = m.update(item_properties=properties)
    
    return update

### FEATURE LAYER OVERWRITE

In [81]:
#get feature layer containing updated data for maps associated with the COVID-19 vaccines
layer = gis.content.get(vacc_layer_id)
layer

layer_collection = FeatureLayerCollection.fromitem(layer)

#call the overwrite() method which can be accessed using the manager property
layer_collection.manager.overwrite(path_out)

{'success': True}

### UPDATE MAP SYMBOLOGY

In [82]:
#get map data
vacc1_data = get_map(vacc_count_map_id)

#adjust symbology for graduated points to reflect new max/min
#MAX VACC COUNT
vacc1_data['operationalLayers'][2]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_count.item()
vacc1_data['operationalLayers'][2]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_count.item()

vacc1_data['operationalLayers'][3]['layerDefinition']['drawingInfo']['renderer']['authoringInfo']['visualVariables'][0]['maxSliderValue'] = max_count.item()
vacc1_data['operationalLayers'][3]['layerDefinition']['drawingInfo']['renderer']['visualVariables'][0]['maxDataValue'] = max_count.item()

#update map to save changes
vacc1_update = update_map(vacc_count_map_id, vacc1_data)
vacc1_update

<Item title:"SD COVID-19 Vaccination Counts" type:Web Map owner:jembury8568_SDSUGeo>


True

### VIEW UPDATED DASHBOARD

In [153]:
webbrowser.open(vacc_dash, new=2)

True